The torch.nn package from pytorch contains utility classes for building neural networks.

In [1]:
import torch.nn as nn
import numpy as np
import torch

In [25]:
#Input (temp,rainfall,humidity)

inputs=np.array([[73,67,43],[91,88,64],[87,134,58],[102,43,37],[69,96,70],[73,67,43],
                [91,88,64],[87,134,58],[102,43,37],[69,96,70],[73,67,43],[91,88,64],
                [87,134,58],[102,43,37],[69,96,70]],dtype="float32")

# Targets (apples and oranges)
targets= np.array([[56,70],[81,101],[119,133],[22,37],[10,119],[56,70],[81,101]
                  ,[119,34],[22,56],[103,110],[56,90],[81,90],[119,89],[22,45],[102,119]],dtype="float32")
inputs=torch.from_numpy(inputs)
targets=torch.from_numpy(targets)

- Here I am using 15 training examples

#### Dataset and DataLoader

Here TensorDataset is created which allows acces to rows from inputs and targets as tuples and provides standard APIs for working with many different
types of datasets in PyTorch.

In [3]:
from torch.utils.data import TensorDataset

In [28]:
train_ds=TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

#### Now I will create a DataLoader which can split the data into batches of predefined size while training.

- We are using batching because if we are loading 1000 of data to memory it will create computational challenges to system, so to avoid it we can use batches and send to memory for computation.

In [5]:
from torch.utils.data import DataLoader

In [29]:
# Define Data Loader
batch_size=5
train_dl= DataLoader(train_ds,batch_size,shuffle=True)

The data loader is typically used in a for-in loop.

In [7]:
for xb,yb in train_dl:
    print("Batches:")
    print(xb)
    print(yb)
    break

Batches:
tensor([[102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 73.,  67.,  43.],
        [ 87., 134.,  58.],
        [ 73.,  67.,  43.]])
tensor([[ 22,  56],
        [ 81,  90],
        [ 56,  70],
        [119, 133],
        [ 56,  90]], dtype=torch.int32)


### Since shuffle is set to True, it shuffles the training data before creating batches.
- Shuffle helps randomize the input to the optimization algorithm, which can lead to faster reduction in loss.

#### nn.Linear

- Instead of intializing the weights and biases manually, we can define the model using teh nn.Linear whcih does int automatically.

In [30]:
## Define model
model=nn.Linear(3,2) # 3 is the number of inputs and 2 is the number of outputs
print(model.weight) # Model itself creates weights 
print(model.bias)   # Model itself creates biases

Parameter containing:
tensor([[-0.3925, -0.0574,  0.3230],
        [-0.5157, -0.1823, -0.3197]], requires_grad=True)
Parameter containing:
tensor([0.1249, 0.3237], requires_grad=True)


- Pytorch model have a helpful parameter called .parameters method, which returns  a list containing all the weights and matrices present in the model.

- For Linear regression model, we have one weight matrix and one bias matrix.

In [31]:
# Parameters

list(model.parameters())

[Parameter containing:
 tensor([[-0.3925, -0.0574,  0.3230],
         [-0.5157, -0.1823, -0.3197]], requires_grad=True),
 Parameter containing:
 tensor([0.1249, 0.3237], requires_grad=True)]

In [32]:
#Generating Predictions
preds=model(inputs)
print(preds)

tensor([[-18.4871, -63.2803],
        [-19.9750, -83.1037],
        [-22.9839, -87.5067],
        [-30.4307, -71.9432],
        [ -9.8606, -75.1346],
        [-18.4871, -63.2803],
        [-19.9750, -83.1037],
        [-22.9839, -87.5067],
        [-30.4307, -71.9432],
        [ -9.8606, -75.1346],
        [-18.4871, -63.2803],
        [-19.9750, -83.1037],
        [-22.9839, -87.5067],
        [-30.4307, -71.9432],
        [ -9.8606, -75.1346]], grad_fn=<AddmmBackward>)


### Loss Function

- Instead of defining a loss function manually, we can use built-in loss function mse_loss.

In [11]:
## nn.functional has lots of loss functions,error functions and several other utilitties.
import torch.nn.functional as F

In [33]:
#we can use mse_loss to find the measn squared error loss
loss_fn=F.mse_loss

In [43]:
loss=loss_fn(model(inputs),targets)
print(loss)

tensor(411.3709, grad_fn=<MseLossBackward>)


### Optimizer

- Instead of manually manipualting the model's weights and biases using gradients, we can use the optimizer optim.SGD. SGD stands for stochastic gradient descent. It is alled stochastic becasue samples are selected in batches(often with random shuffling) instead of a single group.

In [34]:
#Defining Optimizer

opt=torch.optim.SGD(model.parameters(),lr=1e-5)

Note that model.parameters() is passed as an argument to optim.SGD, so that optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate which controls the amount by which parameters
are modified.

In [44]:
#Utility function to train the model.
def fit(num_epochs,model,loss_fn,opt,train_dl):
    for epoch in range(num_epochs):
        for xb,yb in train_dl:
            pred=model(xb)
            loss=loss_fn(pred,yb)
            loss.backward() #compute gradients
            opt.step() #updating parameters using gradients
            opt.zero_grad() #reseting the gradient to zero
            
            #printing the progress
            
            if (epoch+1)%10==0:
                print("Epoch [{}/{}],Loss {:.4f}".format(epoch+1,num_epochs,loss.item() ))

In [48]:
fit(500,model,loss_fn,opt,train_dl)

Epoch [10/500],Loss 407.5816
Epoch [10/500],Loss 758.2211
Epoch [10/500],Loss 124.6696
Epoch [20/500],Loss 512.7873
Epoch [20/500],Loss 191.4151
Epoch [20/500],Loss 549.3286
Epoch [30/500],Loss 81.0687
Epoch [30/500],Loss 880.6526
Epoch [30/500],Loss 328.6092
Epoch [40/500],Loss 289.9973
Epoch [40/500],Loss 904.0948
Epoch [40/500],Loss 91.7841
Epoch [50/500],Loss 106.1917
Epoch [50/500],Loss 561.4618
Epoch [50/500],Loss 573.0817
Epoch [60/500],Loss 267.1618
Epoch [60/500],Loss 427.7354
Epoch [60/500],Loss 608.8589
Epoch [70/500],Loss 145.0195
Epoch [70/500],Loss 85.4476
Epoch [70/500],Loss 1027.8165
Epoch [80/500],Loss 708.2224
Epoch [80/500],Loss 437.5897
Epoch [80/500],Loss 147.6322
Epoch [90/500],Loss 499.5178
Epoch [90/500],Loss 449.1245
Epoch [90/500],Loss 325.8586
Epoch [100/500],Loss 147.2988
Epoch [100/500],Loss 725.2679
Epoch [100/500],Loss 422.7929
Epoch [110/500],Loss 320.0811
Epoch [110/500],Loss 365.5114
Epoch [110/500],Loss 593.3062
Epoch [120/500],Loss 375.7444
Epoch [12

In [51]:
preds=model(inputs)
preds

tensor([[ 54.5147,  65.1891],
        [ 70.4950, 101.0628],
        [120.6551,  87.5892],
        [ 27.5420,  49.1075],
        [ 79.8919, 116.6575],
        [ 54.5147,  65.1891],
        [ 70.4950, 101.0628],
        [120.6551,  87.5892],
        [ 27.5420,  49.1075],
        [ 79.8919, 116.6575],
        [ 54.5147,  65.1891],
        [ 70.4950, 101.0628],
        [120.6551,  87.5892],
        [ 27.5420,  49.1075],
        [ 79.8919, 116.6575]], grad_fn=<AddmmBackward>)

In [53]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [ 10., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119.,  34.],
        [ 22.,  56.],
        [103., 110.],
        [ 56.,  90.],
        [ 81.,  90.],
        [119.,  89.],
        [ 22.,  45.],
        [102., 119.]])

In [54]:
import jovian

In [55]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/nithin212/pytorch-linear-regression-using-buitlins


'https://jovian.ml/nithin212/pytorch-linear-regression-using-buitlins'